In [ ]:
# !pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [38]:
import os
from PIL import Image
import numpy as np
import cv2

In [58]:
data_dir = '/home/jems/cmsc422/final/samples'  # Replace with the actual path to your data
image_size = (80, 60)

X = []  # Array to store images
y = []  # Array to store sample labels

for sample_id in range(1, 63):  # Replace 100 with the total number of samples
    sample_dir = os.path.join(data_dir, f'Sample{sample_id:03d}')
    
    for image_file in os.listdir(sample_dir):
        if image_file.endswith('.jpg') or image_file.endswith('.png'):  # Adjust as needed
            image_path = os.path.join(sample_dir, image_file)
            # Load the image
            image = Image.open(image_path)
            # Resize the image
            image = image.resize(image_size)
            # Convert the image to a numpy array and convert to grayscale
            image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)  # Using OpenCV for grayscale conversion
            # Append the grayscale image to X
            X.append(image)
            # Append the label to y (could be sample_id or another way to represent the label)
            y.append(sample_id)

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Reshape X to match the input layer dimensions for the ANN
X = X.reshape(-1, 4800)  # 80 * 60 = 4800

In [59]:
print(X.shape)

(3410, 4800)


In [60]:
class ANN():

    def __init__(self):
        self.inputSize = 4800
        self.outputSize = 62
        self.hiddenSize = 100
        self.learning_rate = None

        self.hidden = np.random.randn(self.inputSize, self.hiddenSize) * 0.01
        self.output = np.random.randn(self.hiddenSize, self.outputSize) * 0.01
        self.hidden_biases = np.ones((1, self.hiddenSize))
        self.output_biases = np.ones((1, self.outputSize))
    
    def sigmoid(self, s):
        return 1/(1 + np.exp(-s))

    def forward(self, X):
        self.z1 = np.dot(X, self.hidden) + self.hidden_biases
        self.A1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.A1, self.output) + self.output_biases
        self.A2 = self.sigmoid(self.z2)
        return self.A2    
    
    def backpropogation(self, X, y, learning_rate):
        def sig_derivative(x):
            return x * (1 - x)
        
        
        self.learning_rate = learning_rate
        m = X.shape[0]
        
        dA2 = (self.A2 - y) * (2/m)

        # Gradients for the output layer
        dZ2 = dA2 * sig_derivative(self.A2)
        dW2 = np.dot(self.A1.T, dZ2)
        db2 = np.sum(dZ2, axis=0, keepdims=True)

        # Gradients for the hidden layer
        dA1 = np.dot(dZ2, self.output.T)
        dZ1 = dA1 * sig_derivative(self.A1)
        dW1 = np.dot(X.T, dZ1)
        db1 = np.sum(dZ1, axis=0, keepdims=True)

        # Update weights and biases
        self.hidden -= self.learning_rate * dW1
        self.hidden_biases -= self.learning_rate * db1
        self.output -= self.learning_rate * dW2
        self.output_biases -= self.learning_rate * db2

In [ ]:
from collections import defaultdict

sorted = defaultdict(list)

for i in range(len(X)):
    sorted[y[i]].append(X[i])

def split_data(data, label):
    np.random.shuffle(data)
    test = data[:5]
    test_y = np.full((5, 1), label)
    train = data[5:]
    train_y = np.full((len(train), 1), label)
    return train, train_y, test, test_y

train_X = []
train_y = []
test_X = []
test_y = []

for label, data in sorted.items():
    train, train_y_, test, test_y_ = split_data(data, label)
    train_X.extend(train)
    train_y.extend(train_y_)
    test_X.extend(test)
    test_y.extend(test_y_)

train_X = np.array(train_X)
train_y = np.array(train_y)
test_X = np.array(test_X)
test_y = np.array(test_y)

print(train_X.shape)

ann = ANN()

def train_ann(X_train, y_train, epochs=100, learning_rate=0.01):
    # Convert y_train to one-hot encoding
    num_classes = len(np.unique(y_train))
    y_train_onehot = np.zeros((y_train.shape[0], num_classes))
    y_train_onehot[np.arange(y_train.shape[0]), y_train] = 1

    # Create and train the ANN
    ann = ANN()
    ann.train(X_train, y_train_onehot, epochs=epochs, learning_rate=learning_rate)
    return ann

# Testing function
def test_ann(ann, X_test, y_test):
    predictions = ann.predict(X_test)
    accuracy = np.mean(predictions == y_test) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Example usage
# Assuming X_train, X_test, y_train, y_test are already prepared
ann = train_ann(train_X, train_y, epochs=100, learning_rate=0.01)
test_accuracy = test_ann(ann, test_X, test_y)

